In [10]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon")
%run data_prep_10step_5horizon.ipynb
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon")
%run LSTM_Model_Classes.ipynb
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/")
%run Useful_Functions.ipynb



In [27]:
batch_size = 1
lookback = 10
process_embedding = ProcessEmbedding(df2_allhero, embedding_dim=8, lookback=lookback) # we create the embedding vector on unsplit data to ensure all unique hero id's are contained
input_size = process_embedding.embedding_dim + 1 #84 + 1
hidden_size = 64
num_layers = 2
output_size = 5  # Assuming you want to predict 5 values
model = LSTMModel(input_size, hidden_size, num_layers, output_size, process_embedding)


In [28]:
train_dataset = TimeSeriesDataset(df2_70, lookback=lookback)
test_dataset = TimeSeriesDataset(df2_70_remain, lookback=lookback)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))



#Load the pre-trained model
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/lstm")
model.load_state_dict(torch.load('10step_5horizon_RMSE_8.pth'))


<All keys matched successfully>

Calculate training loss RMSE

In [29]:
model.eval()
train_loss = 0.0
with torch.no_grad():
  for batch in train_loader:
    hero_ids, X, y = batch
    X = X[0]
    y = y[0]
    outputs = model(batch)
    targets = y[:, -5:]
    loss = nn.MSELoss()(outputs.squeeze(), targets.squeeze())
    train_loss += loss.item() * X.size(0)
    train_loss /= len(train_dataset)
    train_rmse = np.sqrt(train_loss)

print(f"Unscaled Training RMSE: {ConstantUnScaler(train_rmse, min_gold = min_gold, max_gold = max_gold)}")


Unscaled Training RMSE: 44.0


Calculate test loss RMSE

In [30]:
test_loss = 0.0
with torch.no_grad():
  for batch in test_loader:
    hero_ids, X, y = batch
    X = X[0]
    y = y[0]
    outputs = model(batch)
    targets = y[:, -5:]
    loss = nn.MSELoss()(outputs.squeeze(), targets.squeeze())
    test_loss += loss.item() * X.size(0)
    test_loss /= len(test_dataset)
    test_rmse = np.sqrt(test_loss)
print(f"Unscaled Test RMSE: {ConstantUnScaler(test_rmse, min_gold = min_gold, max_gold = max_gold)}")

Unscaled Test RMSE: 138.0


Create a DataFrame with the metrics


In [25]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/lstm')

In [31]:
metrics_df = pd.DataFrame({'Metric': ['Training RMSE', 'Test RMSE'],
'Value': [ConstantUnScaler(train_rmse, min_gold = min_gold, max_gold = max_gold), ConstantUnScaler(test_rmse, min_gold = min_gold, max_gold = max_gold)]})
#Save the metrics to a CSV file

metrics_file = "10step_5horizon_RMSE_8.csv"
metrics_df.to_csv(metrics_file, index=False)
print(f"Metrics saved to {metrics_file}\n{metrics_df}")

Metrics saved to 10step_5horizon_RMSE_8.csv
          Metric  Value
0  Training RMSE   44.0
1      Test RMSE  138.0
